# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
#Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key 

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = "/Users/tattwamasiray/Downloads/Monash-BOOTCAMP/Python-API-Challenge/Starter_Code/output_data/cities.csv"
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:


# Configure the map plot


# Display the map

city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', 
                             colorbar=True, title='City Humidity', 
                             xlabel='Longitude', ylabel='Latitude',
                             hover_cols=['City'],
                             size=city_data_df['Humidity']*10,
                             alpha=0.5)


:Scatter   [Lng]   (Lat,Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# # Narrow down cities that fit criteria and drop any results with null values



# # Drop any rows with null values



# # Display sample data

ideal_cities_df = city_data_df[(city_data_df['Max Temp'] >= 70) &
                               (city_data_df['Max Temp'] <= 80) &
                               (city_data_df['Wind Speed'] < 10) &
                               (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df.head()



/var/folders/7c/fj34rrws5f518q6t6d9dxnpm0000gn/T/ipykernel_17762/765068656.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# display sample data
print(hotel_df.head())


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# display sample data
print(hotel_df.head())




          City Country      Lat       Lng  Humidity
0         faya      SA  18.3851   42.4509        35
1      farsund      NO  58.0948    6.8047       100
2  new norfolk      AU -42.7826  147.0587        58
3    jamestown      US  42.0970  -79.2353        77
4      lanzhou      CN  36.0564  103.7922        48
          City Country      Lat       Lng  Humidity Hotel Name
0         faya      SA  18.3851   42.4509        35           
1      farsund      NO  58.0948    6.8047       100           
2  new norfolk      AU -42.7826  147.0587        58           
3    jamestown      US  42.0970  -79.2353        77           
4      lanzhou      CN  36.0564  103.7922        48           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:

radius = 10000
params = {
    "type": "hotel",
    "radius": radius,
    "sort": "distance",
    "limit": 1,
    "apiKey": "522f505670bf45869318b1ddd9a2f652"
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude, and country code from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    country_code = row["Country"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"countrycode:{country_code}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v1/places"
    
    # Send request to Geoapify API and extract hotel data
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Extract hotel name from response and add to DataFrame
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.at[index, "Hotel Name"] = hotel_name
        print(f"Found hotel near {lat}, {lng}: {hotel_name}")
    except:
        print(f"No hotels found near {lat}, {lng}")

# Display sample data
hotel_df.head()


No hotels found near 18.3851, 42.4509
No hotels found near 58.0948, 6.8047
No hotels found near -42.7826, 147.0587
No hotels found near 42.097, -79.2353
No hotels found near 36.0564, 103.7922
No hotels found near -54.8, -68.3
No hotels found near 42.6001, -73.9662
No hotels found near -24.8667, 113.6333
No hotels found near 66.9395, -53.6735
No hotels found near -21.3393, 55.4781
No hotels found near -22.8375, -51.9731
No hotels found near 59.7035, 30.7861
No hotels found near -3.2778, -39.2689
No hotels found near 58.4619, 56.4165
No hotels found near 77.484, -69.3632
No hotels found near -46.1927, 168.8643
No hotels found near 71.9667, 102.5
No hotels found near 65.6145, -37.6368
No hotels found near 71.6872, 128.8694
No hotels found near -0.7393, -90.3518
No hotels found near -21.2078, -159.775
No hotels found near 27.4763, -99.5164
No hotels found near -16.3494, -53.7583
No hotels found near -23.1203, -134.9692
No hotels found near 62.55, 155.55
No hotels found near -35.3333, -72.4

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


Add the hotel name and the country as additional information in the hover messa### Step 5: ge for each city in the map.

In [6]:
# Set up the plot
fig = go.Figure()

# Add a scattermapbox trace with the city data
fig.add_trace(go.Scattermapbox(
    lat=city_data_df["Lat"],
    lon=city_data_df["Lng"],
    mode="markers",
    marker=go.scattermapbox.Marker(
        size=city_data_df["Humidity"],
        color=city_data_df["Humidity"],
        colorscale="Viridis",
        opacity=0.7
    ),
    text=city_data_df["City"],
    hoverinfo="text"
))

# Add a scattermapbox trace with the hotel data
fig.add_trace(go.Scattermapbox(
    lat=hotel_df["Lat"],
    lon=hotel_df["Lng"],
    mode="markers",
    marker=go.scattermapbox.Marker(
        size=10,
        color="red"
    ),
    text=hotel_df["Hotel Name"] + ", " + hotel_df["Country"],
    hoverinfo="text"
))

# Set the layout
fig.update_layout(
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_token,
        center=go.layout.mapbox.Center(
            lat=center_lat,
            lon=center_lon
        ),
        zoom=zoom
    ),
    margin={"l": 0, "r": 0







SyntaxError: f-string: expecting '}' (480204154.py, line 3)